In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import json
import torch
import random
import tokenize
import numpy as np
from tqdm import tqdm
import multiprocessing
from torch.utils.data import DataLoader, Dataset, RandomSampler
from transformers import RobertaConfig, RobertaForMaskedLM, RobertaTokenizer
from transformers import WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from parser import remove_comments_and_docstrings

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from transformers import RobertaConfig, RobertaForMaskedLM, RobertaTokenizer
from transformers import WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import recall_score, precision_score, f1_score

In [2]:
class arguments(object):
    def __init__(self):
        pass
args = arguments()

In [3]:
args.total_length = 512
args.graph_length = 0
args.epochs = 10

args.topic_num = 12

# args.train_batch_size = 1
# args.eval_batch_size = 1

args.gradient_accumulation_steps = 1
args.max_grad_norm = 1.0
args.learning_rate = 5e-5
args.weight_decay = 0.0
args.adam_epsilon = 1e-8

args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.seed = 978438233

def set_seed():
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)
set_seed()

In [4]:
config = RobertaConfig.from_pretrained('microsoft/codebert-base')
config.num_labels = 1
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')
model0 = RobertaForMaskedLM.from_pretrained('microsoft/codebert-base', config = config)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def get_tokens(code, do_remove, tmp_file_name):
    if (do_remove):
        code = remove_comments_and_docstrings(code, 'python')
    output_file = open(tmp_file_name, 'w')
    print(code, file = output_file)
    output_file.close()
    
    tokens = []
    f = open(tmp_file_name, 'rb')
    tokenGenerator = tokenize.tokenize(f.readline)
    for token in tokenGenerator:
        if (token.type in [0, 59, 60, 62]): # COMMENT
            pass
        elif (token.type in [4, 61]): # NEWLINE
            pass
        elif (token.type == 5): # INDENT
            pass
        elif (token.type == 6): # DEDENT
            pass
        elif (token.type in [1, 2, 3, 54]): # NAME NUMBER STRING OP
            tokens.append(token.string)
        else:
            assert(False)
    f.close()
    return tokens

def search(path):
    data = []
    if (os.path.isdir(path)):
        for filename in os.listdir(path):
            data.extend(search(path + '/' + filename))
    else:
        assert(os.path.isfile(path))
        input_file = open(path, 'r')
        code = input_file.read()
        tmp_file_name = path.replace('/', '---').replace('.', '') + '.py'
        tmp_file_name = 'tmp2.py'
        try:
            tokens = get_tokens(code, True, tmp_file_name)
        except:
            tokens = get_tokens(code, False, tmp_file_name)
        os.system('rm ./' + tmp_file_name)
        if (len(tokens) != 0):
            tokens = [tokenizer.tokenize(tokens[0])] \
                   + [tokenizer.tokenize('@ ' + x)[1 :] for x in tokens[1 :]]
            tokens = [y for x in tokens for y in x]
            tokens = tokens[: args.total_length - 2]
            
            code_ids = [tokenizer.cls_token_id] + tokenizer.convert_tokens_to_ids(tokens)
            position_ids = [i + tokenizer.pad_token_id + 1 for i in range(len(code_ids))]
            padding_length = args.total_length - len(code_ids)
            code_ids += [tokenizer.pad_token_id] * padding_length
            position_ids += [tokenizer.pad_token_id] * padding_length
            
            data.append((code_ids, position_ids))
        input_file.close()
    return data

topic_map = dict()

def read_data(repo_file, topic_file):
    dataset = []
    f = open(repo_file, 'r')
    repos = []
    for line in f:
        _, repo = line.strip().split(chr(9))
        repo = repo[repo.rfind('/', 0, repo.rfind('/') - 1) + 1 :]
        repos.append(repo)
    f.close()
    
#     repos = repos[2010:]
#     repos = repos[:100] # TODO

    for repo in tqdm(repos, total = len(repos)):
        data = search('../data/py150_files/' + repo)
        dataset.append(([x for x, y in data], [y for x, y in data]))
    return dataset

dataset = read_data('../data/py150/github_repos.txt', '../data/py150/repo_topics2.jsonl')

100%|█████████████████████████████████████| 8422/8422 [1:06:11<00:00,  2.12it/s]


In [6]:
f = open('dataset.jsonl', 'w')
json.dump({'dataset' : dataset}, fp = f)
f.close()